In [ ]:
%load_ext autoreload
%autoreload 2

Set things up

In [ ]:
from src.tools.sck import sck
from src.saf import *
import time
from bokeh.models.sources import ColumnDataSource
from bokeh.plotting import figure
from bokeh.io import output_notebook, show, push_notebook
from threading import Thread

output_notebook()

Make the kit

In [ ]:
kit = sck()
kit.begin()

Some things we can know about this kit

In [ ]:
print (kit.sensor_enabled)
print (kit.serialPort)
print (kit.serialPort_name)
print (kit.sam_firmVer)
print (kit.sam_firmCommit)
print (kit.sam_firmBuildDate)
print (kit.esp_macAddress)

We can set shell mode for easier interaction

In [ ]:
kit.toggleShell()

Here we can define what channels we would like to see, and some processing for them.
<br>Each channel can have several processing to do:

In [ ]:
channels_to_monitor = {'Temperature': {'smooth': 10, 
                                       'time_derivative': 1},
                       'Humidity': {'time_derivative': None}
                      }

In [ ]:
store_to_csv = True
if store_to_csv: path_to_store = join(getcwd(), 'csv_export.csv'); print (path_to_store)
# And start monitoring them
kit.monitor(list(channels_to_monitor.keys()), noms = True)

### Time series plot

In [ ]:
def process_data(data):
    for channel in channels_to_monitor.keys():
        for process in channels_to_monitor[channel].keys():
            if data.empty: data[channel + '_' + process] = []
            else:
                formula = process + f"(new_data['{channel}'], channels_to_monitor['{channel}']['{process}'])"
                data[channel + '_' + process] = eval(formula)
    return data

test_data = ColumnDataSource(data = process_data(kit.worker.example))

from bokeh.models import Panel, Tabs
from bokeh.layouts import column
from bokeh.plotting import figure
from bokeh.io import show
from bokeh.layouts import gridplot

n_tabs = len(list(channels_to_monitor.keys()))
tabs = Tabs(tabs = [])

for channel in channels_to_monitor.keys():
    gridplots = list()
    p = figure(background_fill_color="#fafafa", x_axis_type='datetime')
    list_channels = [channel]
    
    if 'smooth' in channels_to_monitor[channel].keys(): list_channels.append(channel + '_' + 'smooth')
    print (list_channels)
    p.scatter(y=list_channels[0], x="Time", source = test_data)
    gridplots.append([p])
    print (gridplots)
    for process in channels_to_monitor[channel].keys():
        if process != 'smooth':
            p = figure(background_fill_color="#fafafa", x_axis_type='datetime')
            print (channel + '_' + process)
            p.scatter(y=channel + '_' + process, x="Time", source = test_data)
            gridplots.append([p])
    
    grid = gridplot(gridplots, plot_width=600, plot_height=400)
    tab = Panel(child=grid, title=channel)
    tabs.tabs.append(tab)

handle = show(tabs, notebook_handle=True)
    
stop_threads = False

def worker_call(id, stop):
    
    period = .1  # in seconds (simulate waiting for new data)
    n_show = 10  # number of points to keep and show

    while True:
        if not kit.worker.output.empty():
            new_data = kit.worker.output.get()
            if 'Time' in new_data.columns: new_data.set_index('Time')
            
            # Add processing
            process_data (new_data)

            test_data.stream(new_data, n_show)
            # Store to csv
            if store_to_csv: 
                dataframe = pd.DataFrame(test_data.data)
                dataframe.to_csv(path_to_store, sep=",")
            
            # Update plot
            push_notebook(handle=handle)
            time.sleep(period)

            if stop():
                print("Finished thread")
                break

thread = Thread(target=worker_call, args=(id, lambda: stop_threads))
thread.start()

In [ ]:
stop_threads = True